<a href="https://colab.research.google.com/github/AICFT-Miyata/AICFT/blob/main/semantic_search_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#💻 準備段階
この段階では、必要なライブラリのインストール、Google ColabとGoogle Driveの認証、定数の設定、および意味検索モデルとGGUFモデルのロードを行います。

#1. ライブラリのインストールとインポート
この部分は、システムが動作するために必要なすべての依存関係をインストールします。

In [1]:
# --- 1. 必要なライブラリのインストールとインポート ---
# 警告が出ますが、無視して処理を続行してください。

# 1. 意味検索と言語処理用ライブラリ
!pip install -qq fugashi ipadic unidic-lite sentence-transformers scikit-learn gspread

# 2. GGUF実行に必要なllama-cpp-pythonをGPUサポート付きでインストール
# ⚠️ インストールには時間がかかり、途中文字化けのような出力が出ますが、処理を中断しないでください。
!CMAKE_ARGS="-DLLAMA_CUDA=on" FORCE_CMAKE=1 pip install llama-cpp-python

# --- ライブラリのインポート ---
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import gspread
import time
import google.auth
from google.colab import auth

# GGUF用ライブラリ（ここでLlamaクラスを定義）
try:
    from llama_cpp import Llama
    print("✅ Llama-cpp-pythonのインポートに成功しました。")
except ImportError:
    print("❌ Llama-cpp-pythonのインポートに失敗しました。AI洗練機能は動作しません。")
    Llama = None # エラーハンドリングのためLlamaをNoneに設定

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 94.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 694.9/694.9 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 12.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.1 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl size=52058549 sha256=e54b6d792127cfef62556e63b33e3c27786bdb193be326f5fcb4473ea015ded0
  Stored in directory: /root/.cache/pip/wheels/90/82/ab/8784ee3fb99ddb07fd36a679ddbe63122cc07718f6c1eb3be8
Successfully built llama-cpp-python
✅ Llama-c

In [3]:
import tensorflow as tf
print("TensorFlow detected GPU:", tf.config.list_physical_devices('GPU'))

# または、より詳細な情報を確認したい場合
!nvidia-smi

TensorFlow detected GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Thu Dec  4 10:36:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |     

もし上記のコマンドでGPUが検出されない場合、Colabのメニューから「ランタイム」->「ランタイムのタイプを変更」->「ハードウェアアクセラレータ」を「GPU」に設定して、再度ランタイムを接続し直してください。

#2. 定数・パスの定義とモデルの初期化
この部分は、システムが使用するファイルパス、スプレッドシート名、およびAIモデルをメモリにロードする処理です。

⚠️ 注意事項: 以下のコードを実行する前に、SPREADSHEET_NAME、TARGET_SHEET、TRAINING_DATA_PATH、GGUF_MODEL_PATHの4つの変数をご自身のGoogle Drive/スプレッドシートのパスに修正する必要があります。

In [2]:
# --- 2. 定数・初期設定 ---
# ⚠️ 以下の3つのパス・名前をあなたの環境に合わせて最終修正してください
SPREADSHEET_NAME = 'スプレッドシート作成検証'
TARGET_SHEET = '検索入力シート'
TRAINING_DATA_PATH = '/content/drive/MyDrive/AICFT/DB.csv'

# ⚠️ GGUFモデルのパスを、あなたがアップロードした Q5_K_M のものに修正してください
GGUF_MODEL_PATH = '/content/drive/MyDrive/AICFT/LLM_GGUF/llama3-8b-instruct-Q5_K_M.gguf'

# モデルのロード (意味検索用)
EMBEDDING_MODEL = SentenceTransformer('cl-tohoku/bert-base-japanese-whole-word-masking')

# GGUFモデルの初期設定 (AI洗練用)
LLAMA_CLIENT = None
if Llama:
    try:
        # モデルのロード（メモリ効率が良いQ5_K_MをGPUの全レイヤーで実行）
        LLAMA_CLIENT = Llama(
            model_path=GGUF_MODEL_PATH,
            n_ctx=4096,
            n_gpu_layers=-1,
            verbose=False
        )
        print("✅ GGUFモデルのロードに成功しました。")
    except Exception as e:
        print(f"❌ GGUFモデルのロードに失敗しました。AI洗練機能は動作しません。エラー: {e}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

llama_context: n_ctx_per_seq (4096) < n_ctx_train (8192) -- the full capacity of the model will not be utilized


✅ GGUFモデルのロードに成功しました。


#🚀 実行段階
この段階では、定義された関数を実行し、実際に人事評価のコメントを検索・生成・スプレッドシートへの書き込みを行います。

#3. データ処理とエンベディング生成
訓練データを読み込み、評価コメント検索の基礎となる**意味ベクトル（エンベディング）**を生成します。

In [3]:
# --- 3. データの準備とエンベディング生成 ---
def load_and_embed_data(file_path):
    # ファイル読み込みと列名の整理
    try:
        df_train = pd.read_csv(file_path, encoding='utf-8', sep=',',
             usecols=['階級', '目標（50～100文字）', '自己申告（50～100文字）', '上司コメント（50～100文字）'])
        df_train.columns = ['Rank', 'Goal', 'SelfReport', 'SupervisorComment']
        # 検索用テキストの結合
        df_train['CombinedText'] = df_train['Goal'].astype(str) + " [SEP] " + df_train['SelfReport'].astype(str)
        # エンベディングの生成
        embeddings = EMBEDDING_MODEL.encode(df_train['CombinedText'].tolist(), convert_to_tensor=True)
        return df_train, embeddings.cpu().numpy()
    except Exception as e:
        print(f"データ処理エラー: {e}")
        return None, None

df_train, train_embeddings = load_and_embed_data(TRAINING_DATA_PATH)

#4. 検索ロジックの定義
入力された目標と自己申告に基づき、訓練データから最も類似度の高い（コサイン類似度が高い）既存の上司コメントを検索します。

In [4]:
# --- 4. 検索ロジック ---
def find_best_match_comment(input_goal, input_self_report):
    if df_train is None:
        return "ERROR: Training data not loaded.", 0.0

    # 入力データのエンベディング生成
    new_combined_text = str(input_goal) + " [SEP] " + str(input_self_report)
    new_embedding = EMBEDDING_MODEL.encode(new_combined_text, convert_to_tensor=True).cpu().numpy().reshape(1, -1)

    # コサイン類似度の計算
    similarities = cosine_similarity(new_embedding, train_embeddings)[0]

    # ベストマッチのコメントを取得
    best_match_index = np.argmax(similarities)
    best_comment = df_train.iloc[best_match_index]['SupervisorComment']
    return best_comment, similarities[best_match_index]

#5. AIによるコメント洗練ロジックの定義
検索で見つかったコメント（原案）を基に、LLaMA 3 GGUFモデルを使用して、より具体的で入力データに合ったコメントに洗練（リライト）します。

In [32]:
# --- 5. AIによるコメント洗練ロジック (GGUF実行) ---
def refine_comment_with_ai(input_goal, input_self_report, best_comment):
    if LLAMA_CLIENT is None:
        return best_comment # LLMが動作しない場合は原案をそのまま返す

    try:
        print("--- AIによるコメント洗練開始 (LLaMA 3 GGUF) ---")

        # 日本語での出力形式を強力に指定
        system_prompt = "あなたは人事評価の上司コメントを作成するAIです。日本語で、部下に対する口調で、、定量的な申告に対しては確実に評価し、具体的で建設的なフィードバックを、簡潔に作成してください。"

        user_prompt = f"""
        [入力データ]
        目標: {input_goal}
        自己申告: {input_self_report}
        [推薦コメント (参考)]
        {best_comment}

        上記のデータを参照し、日本語で上司コメントを生成してください。出力はコメント本文のみとしてください。
        """

        # 応答開始を明確に日本語の形式で始めるよう強制
        prompt = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n{system_prompt}<|eot|><|start_header_id|>user<|end_header_id|>\n{user_prompt}<|eot|><|start_header_id|>assistant<|end_header_id|>\n**【上司コメント】**"

        # 推論実行
        output = LLAMA_CLIENT.create_completion(
            prompt=prompt,
            max_tokens=256,
            temperature=0.4, # 創造性を少し下げ、指示に忠実になるように調整
            stop=["<|eot|>"]
        )

        # 応答から強制開始文字列を除去
        refined_comment = output['choices'][0]['text'].replace('**【上司コメント】**', '').strip()
        print("AI洗練コメント生成成功。")
        return refined_comment

    except Exception as e:
        print(f"AI洗練処理エラー。通常通り検索結果を出力します。エラー: {e}")
        return best_comment

#6.メイン処理

In [35]:
# --- 6. メイン処理 (Apps Scriptから実行される想定) ---
def main_search_and_write():
    if df_train is None:
        print("検索スキップ: 訓練データなし")
        return "検索スキップ: 訓練データなし", 0.0

    print("--- gspread認証開始 ---")
    # Colabでの認証
    auth.authenticate_user()
    creds, _ = google.auth.default()
    gc = gspread.authorize(creds)

    spreadsheet = gc.open(SPREADSHEET_NAME)
    input_sheet = spreadsheet.worksheet(TARGET_SHEET)

    # スプレッドシートから入力データを取得
    data_list = input_sheet.get('A2:B2', value_render_option='UNFORMATTED_VALUE')

    if not data_list or len(data_list[0]) < 2:
        print("ERROR: スプレッドシートのA2～B2範囲にデータが不足しています。処理を中断します。")
        return "ERROR: データ不足", 0.0

    input_data = data_list[0]
    input_goal = input_data[0]         # A2 (目標)
    input_self_report = input_data[1]  # B2 (自己申告)

    print(f"入力データ: 目標='{input_goal}', 自己申告='{input_self_report}'")

    # 1. 意味検索実行
    best_comment, similarity_score = find_best_match_comment(input_goal, input_self_report)
    print(f"検索結果 (原案): {best_comment}")

    # 2. AI洗練 (LLM連携)
    final_comment = refine_comment_with_ai(input_goal, input_self_report, best_comment)

    # 3. 結果をスプレッドシートに出力
    input_sheet.update('C2', [[final_comment]])
    input_sheet.update('D2', [[f"類似度: {similarity_score:.4f}"]])
    print(f"最終結果 (C2) を出力完了。類似度: {similarity_score:.4f}")

    return final_comment, similarity_score

if __name__ == "__main__":
    # 単体テストとしてメイン処理を実行
    main_search_and_write()

--- gspread認証開始 ---
入力データ: 目標='AI活用推進室（仮称）の設置', 自己申告='AI活用推進室（仮称）の人員を選定するとともに業務を選定し、円滑に立ち上げることにより、組織のAI活用推進に貢献することができた。
また、組織のAI活用を約３０％向上させることができた。'
検索結果 (原案): 新しい教育技術への積極的な取り組みを評価する。計画に基づき、AI教官機能の小規模な試験運用を開始すること。
--- AIによるコメント洗練開始 (LLaMA 3 GGUF) ---
AI洗練コメント生成成功。


/tmp/ipython-input-2344169690.py:37: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  input_sheet.update('C2', [[final_comment]])
/tmp/ipython-input-2344169690.py:38: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  input_sheet.update('D2', [[f"類似度: {similarity_score:.4f}"]])


最終結果 (C2) を出力完了。類似度: 0.9465
